In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torchsummaryX import summary as summaryX
from torchsummary import summary

In [38]:
#Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [39]:
#Data Transform
transform = transforms.Compose(
    [   transforms.Pad(2),
        transforms.ToTensor()
    ]
)

In [40]:
#Importing Data
train_dataset = datasets.MNIST(root='./data', train=True, transform= transform, download=True)
test_dataset = datasets.MNIST(root = './data', train=False, transform= transform, download=True)

In [41]:
#Defining Hyperparameters
input_size = 32 *32
random_seed = 69
num_epoch = 50
learning_rate = 0.001
batch_size = 16

In [42]:
#Preprocessing Data
train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size= batch_size)
test_loader = DataLoader(dataset=test_dataset, shuffle=False, batch_size=batch_size)

In [43]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, stride= 1)
        self.conv2 = nn.Conv2d(6, 16, 5, stride= 1)
        self.conv3 = nn.Conv2d(16, 120, 5, stride= 1)
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, 10)
        self.avgPool = nn.AvgPool2d(kernel_size = 2, stride= 2)
        self.tanh = nn.Tanh()


    def forward(self, x):
        x = self.conv1(x)
        x = self.tanh(x)
        x = self.avgPool(x)
        x = self.conv2(x)
        x = self.tanh(x)
        x = self.avgPool(x)
        x = self.conv3(x)
        x = self.tanh(x)
        
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        x = self.tanh(x)
        x = self.fc2(x)
        return x

In [44]:
#Checking the Model
x = torch.randn(64, 1, 32, 32)
model = LeNet5()
output = model(x)
print(model)
summary(model, (1,32,32))
print("output.shape : ",output.shape)

LeNet5(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
  (avgPool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (tanh): Tanh()
)
Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 6, 28, 28]           156
├─Tanh: 1-2                              [-1, 6, 28, 28]           --
├─AvgPool2d: 1-3                         [-1, 6, 14, 14]           --
├─Conv2d: 1-4                            [-1, 16, 10, 10]          2,416
├─Tanh: 1-5                              [-1, 16, 10, 10]          --
├─AvgPool2d: 1-6                         [-1, 16, 5, 5]            --
├─Conv2d: 1-7                            [-1, 120, 1, 1]           48,120
├─Tanh: 1-8                            

In [45]:
#Creating a model 
torch.manual_seed(random_seed)
model = LeNet5().to(device)

In [46]:
#Optimizer and Criterion
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

In [47]:
#Training Loop

n_total_steps = len(train_loader)
for ep in range(num_epoch):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        #Forward Pass
        output = model(images)
        loss = criterion(output, labels)

        #Backward Pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #Printing Output
        if (i + 1) % 100 == 0:
            print(f'Epoch: [{ep + 1} / {num_epoch}], Step: [{i + 1} / {n_total_steps}], Loss: {loss.item():.4f}')

Epoch: [1 / 50], Step: [100 / 3750], Loss: 0.4188
Epoch: [1 / 50], Step: [200 / 3750], Loss: 0.3858
Epoch: [1 / 50], Step: [300 / 3750], Loss: 0.0984
Epoch: [1 / 50], Step: [400 / 3750], Loss: 0.0621
Epoch: [1 / 50], Step: [500 / 3750], Loss: 0.0458
Epoch: [1 / 50], Step: [600 / 3750], Loss: 0.1180
Epoch: [1 / 50], Step: [700 / 3750], Loss: 0.4718
Epoch: [1 / 50], Step: [800 / 3750], Loss: 0.3750
Epoch: [1 / 50], Step: [900 / 3750], Loss: 0.2611
Epoch: [1 / 50], Step: [1000 / 3750], Loss: 0.1140
Epoch: [1 / 50], Step: [1100 / 3750], Loss: 0.2066
Epoch: [1 / 50], Step: [1200 / 3750], Loss: 0.0372
Epoch: [1 / 50], Step: [1300 / 3750], Loss: 0.3704
Epoch: [1 / 50], Step: [1400 / 3750], Loss: 0.0408
Epoch: [1 / 50], Step: [1500 / 3750], Loss: 0.1336
Epoch: [1 / 50], Step: [1600 / 3750], Loss: 0.1469
Epoch: [1 / 50], Step: [1700 / 3750], Loss: 0.1235
Epoch: [1 / 50], Step: [1800 / 3750], Loss: 0.0909
Epoch: [1 / 50], Step: [1900 / 3750], Loss: 0.2094
Epoch: [1 / 50], Step: [2000 / 3750], Lo

In [48]:
num_correct = num_samples = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        batch_size = images.shape[0]
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        _, preds = output.max(1)
        num_correct += (preds == labels).sum()
        num_samples += preds.size(0)    
    
    accuracy = (num_correct.item()/num_samples)*100


    print(f'Accuracy of the LeNet-5: {accuracy} %')
        
    

Accuracy of the LeNet-5: 98.71 %
